In test data set we didn't have some columns so first we need predict the columns values based on available values.

In [50]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Library for Modelling 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

#Library for Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

%matplotlib inline

In [51]:
train_data=pd.read_csv('competitive-data-science-predict-future-sales/sales_train.csv')
test_data=pd.read_csv('competitive-data-science-predict-future-sales/test.csv')

In [52]:
train_data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [53]:
#Changing Negative numbers into Positive
train_data['item_price']=train_data['item_price'].abs()
train_data['item_cnt_day']=train_data['item_cnt_day'].abs()

In [54]:
#Checking if the training data has any Null values or not
null_percent=100*(train_data.isnull().sum()/len(train_data))
print(null_percent)

date              0.0
date_block_num    0.0
shop_id           0.0
item_id           0.0
item_price        0.0
item_cnt_day      0.0
dtype: float64


In [55]:
#Grouping item price and item count per month
train_data=train_data.groupby(['date_block_num','shop_id','item_id']).agg({'item_price':'last','item_cnt_day':'sum'}).reset_index()

#Changing the name of the item_cnt_day to item_cnt_month
train_data=train_data.rename(columns={'item_cnt_day':'item_cnt_month'})
train_data.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_month
0,0,0,32,221.0,6.0
1,0,0,33,347.0,3.0
2,0,0,35,247.0,1.0
3,0,0,43,221.0,1.0
4,0,0,51,127.0,2.0


In [56]:
for feature in train_data.columns:
    if 0 in train_data[feature].unique():
        pass
    else:
        print(feature)
        train_data[feature]=np.log(train_data[feature])

item_price
item_cnt_month


In [57]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(train_data)

train_data = pd.DataFrame(scaler.transform(train_data), columns = train_data.columns)

train_data.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_month
0,-1.536816,-1.983784,-1.706875,-0.649855,2.059322
1,-1.536816,-1.983784,-1.706715,-0.213787,1.032998
2,-1.536816,-1.983784,-1.706394,-0.542350,-0.593688
3,-1.536816,-1.983784,-1.705112,-0.649855,-0.593688
4,-1.536816,-1.983784,-1.703830,-1.185296,0.432636


In [58]:
x_train = train_data.drop(labels=["item_cnt_month", "item_price"], axis=1)

y_train = train_data["item_price"]

In [59]:
x_train.columns

Index(['date_block_num', 'shop_id', 'item_id'], dtype='object')

In predict_date_block_num (ipynb file) we predict the date_block_num values use that to predict the sale price

In [60]:
date_block_num= pd.read_csv("predicted/date_block_num.csv")

test_data = test_data.join(date_block_num)
test_data.head()

,ID,shop_id,item_id,predicted_date_block_num
0,0,5,5037,-1.060742
1,1,5,5320,-0.328165
2,2,5,5233,-1.422938
3,3,5,5232,-1.456473
4,4,5,5268,1.177408


In [61]:
scaler.fit(test_data)

test_data = pd.DataFrame(scaler.transform(test_data), columns = test_data.columns)

test_data.isnull().sum()

ID                          0
shop_id                     0
item_id                     0
predicted_date_block_num    0
dtype: int64

In [62]:
x_test = test_data.drop(labels=["ID"], axis=1)

In [63]:
# rr_model=RandomForestRegressor(n_estimators=5)
# rr_model.fit(x_train,y_train)

# rr_predict=rr_model.predict(x_test)
# print(rr_predict)

In [64]:
#Training the model
linear_model=LinearRegression()
linear_model.fit(x_train,y_train)

#Testing the model
linear_prediction=linear_model.predict(x_test)

#output is the predicted cnt_per_month
print(linear_prediction)

[ 0.09676399 -0.10478825  0.19439706 ...  0.02995926 -0.34791378
  0.46183384]


In [65]:
val = pd.Series(rr_predict)

In [66]:
val.to_csv("predicted/item_price.csv", header = ["predicted_item_price"], index=False)